In [1]:
import json

In [2]:
import json
from datetime import datetime, timedelta,date
import pandas as pd
from functools import reduce
import numpy as np 
import os.path

class ReportTM:
    
    def generate_DF(self,name_file):
        with open('data/'+name_file, encoding='utf-8') as fh:
            data = json.load(fh)

        price_date_name = 'currentPrice_'+str(name_file[:-5])
        Data_products = {
            "id" : [],
            'colorDescription' : [],
            'portrait_url' : [],
            'squarishURL' : [],
            'Stock' : [],
            'isNew' : [],
            price_date_name : [],
            'fullPrice' : [],
            'subtitle' : [],
            'title' : [],
            'date':[]
        }

        for product in data:
            Data_products['id'].append(product['id'])
            Data_products['colorDescription'].append(product['colorDescription'])
            Data_products['portrait_url'].append(product['images']['portraitURL'])
            Data_products['squarishURL'].append(product['images']['squarishURL'])
            Data_products['Stock'].append(product['inStock'])
            Data_products['isNew'].append(product['colorways'][0]['isNew'])
            Data_products[price_date_name].append(product['price']['currentPrice'])
            Data_products['fullPrice'].append(product['price']['fullPrice'])
            Data_products['subtitle'].append(product['subtitle'])
            Data_products['title'].append(product['title'])
            Data_products['date'].append(name_file[:-5])
        
        return pd.DataFrame(Data_products)

    def generate_time_serie(self,*files):
        array_dataframes =[]
        for file in files:
            with open('data/'+file, encoding='utf-8') as fh:
                data = json.load(fh)

            price_date_name = file[:-5]
            Data_products = {
                "id" : [],
                price_date_name : []

            }

            for product in data:
                Data_products['id'].append(product['id'])
                Data_products[price_date_name].append(product['price']['currentPrice'])
            array_dataframes.append(pd.DataFrame(Data_products))

        # merge a la lista de dataframes

        df_final = reduce(lambda left,right: pd.merge(left,right,on='id', how='outer'), array_dataframes)

        return df_final
    
    def check_changes_2days(self, day):
    
        #obtenemos el dia anterior a la fecha dada
        date = datetime.strptime(day, "%Y-%m-%d")
        modified_date = date - timedelta(days=1)
        prev_day = datetime.strftime(modified_date, "%Y-%m-%d")
        #comprobamos que existan archivos para los dos dias
        day_file = day +'.json'
        prev_day_file = prev_day+'.json'
        if os.path.isfile('data/'+day_file) and os.path.isfile('data/'+prev_day_file) :
            df_time = self.generate_time_serie(prev_day_file,day_file)
            df_time['change'] = df_time[prev_day_file[:-5]] - df_time[day_file[:-5]]
            
            return df_time.loc[df_time['change']>0].sort_values(by= 'change',ascending=False)
            #
        else:
            raise ValueError(
                    "Nombres de archivo no validos. "
                        f"Only one file per ZIP: {prev_day_file,day_file}"
                    )
            
    

In [3]:
info = ReportTM()


In [17]:

df = info.generate_time_serie('2020-06-27.json','2020-06-28.json','2020-06-29.json',
                              '2020-06-30.json','2020-07-01.json','2020-07-02.json',
                              '2020-07-03.json','2020-07-04.json','2020-07-05.json',
                              '2020-07-06.json','2020-07-07.json','2020-07-08.json',
                              '2020-07-09.json','2020-07-10.json','2020-07-11.json',
                              '2020-07-12.json','2020-07-13.json','2020-07-14.json',
                             '2020-07-15.json', '2020-07-16.json', '2020-07-17.json')
#df.loc[df.id == '2062d322-fbd5-3ada-918f-c281ffb55c07']
#df = info.generate_time_serie('2020-06-27.json','2020-06-28.json')
df.head()

,id,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,...,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17
0,71886537-bd81-386d-9e92-0081a18ad9a2,2899.0,2899.0,2899.0,2899.0,2899.0,2899.0,2899.0,2899.0,2899.0,...,2899.0,2899.0,2899.0,NaN,NaN,NaN,NaN,NaN,2899.0,NaN
1,1cfe5261-d737-3273-b239-99098d70d0e3,2699.0,2699.0,2699.0,2699.0,2699.0,2699.0,2699.0,2699.0,2699.0,...,2699.0,2699.0,2699.0,2699.0,2699.0,2699.0,NaN,NaN,NaN,NaN
2,040320a0-a6e6-34e8-bea0-d800e3a0b981,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,...,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0,1499.0
3,8373a73a-50b6-3da5-a591-6f76cbc75730,1899.0,1899.0,1899.0,1899.0,NaN,NaN,NaN,NaN,NaN,...,1899.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1899.0,NaN
4,86d99542-576d-3859-9055-709cbaffb49a,1899.0,1899.0,1899.0,1899.0,NaN,NaN,NaN,1899.0,NaN,...,1899.0,1899.0,NaN,1899.0,1899.0,1899.0,1899.0,1899.0,1899.0,1899.0


In [16]:

df.loc[df.id == '98ff2bda-6d09-5f4c-a379-1287bd357590']

,id,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,...,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17


In [7]:
df['test']= df['2020-07-01']- df['2020-07-05']


In [8]:

df = info.generate_time_serie('2020-07-11.json','2020-07-06.json')


In [9]:
df.loc[df['2020-07-06']< 1100]

,id,2020-07-01,2020-07-06
40,92724f14-4f6e-38f4-9c24-3a71dc2057f6,1049.0,1049.0
41,92724f14-4f6e-38f4-9c24-3a71dc2057f6,1049.0,1049.0
81,1e2f4719-8d86-3d9b-adab-5a6ad826da2d,749.0,749.0
157,09efec62-031d-301b-97c3-d3cf90f9ffa5,949.0,949.0
185,6e24fff1-6034-35c7-ab82-10f05c45533d,1099.0,1099.0
258,bc46c52b-8fef-3092-8a4a-c48be3b1874f,599.0,599.0
490,8436d335-09f2-3aa2-a75c-7fef2c5314cf,1078.0,1078.0
529,ee00d6e2-122c-3b30-973c-6284015a5f45,NaN,799.0
553,36a1ad21-464e-356e-bb8a-841889329d5d,NaN,849.0
657,4242af72-1251-301f-8f57-3105c28423a8,NaN,1038.0


In [10]:
info.check_changes_2days('2020-07-23')

,id,2020-07-22,2020-07-23,change


In [38]:
with open('data/Liquidacion/2020-07-08.json', encoding='utf-8') as fh:
         data = json.load(fh)


In [48]:
      
Data_products = {
            "id" : [],
            'colorDescription' : [],
            'portrait_url' : [],
            'squarishURL' : [],
            'Stock' : [],
            'isNew' : [],
            'fullPrice' : [],
            'subtitle' : [],
            'current_price':[],
            'title' : []
}
    
for product in data:
        Data_products['id'].append(product['id'])
        Data_products['colorDescription'].append(product['colorDescription'])
        Data_products['portrait_url'].append(product['images']['portraitURL'])
        Data_products['squarishURL'].append(product['images']['squarishURL'])
        Data_products['Stock'].append(product['inStock'])
        Data_products['isNew'].append(product['colorways'][0]['isNew'])
        Data_products['fullPrice'].append(product['price']['fullPrice'])
        Data_products[ 'current_price'].append(product['price']['currentPrice'])
        Data_products['subtitle'].append(product['subtitle'])
        Data_products['title'].append(product['title'])

In [49]:
Data_  = pd.DataFrame(Data_products)

In [64]:
with open('data/Liquidacion/2020-07-13.json', encoding='utf-8') as fh:
        data2 = json.load(fh)    
Data_products = {
            "id" : [],
            'colorDescription' : [],
            'portrait_url' : [],
            'squarishURL' : [],
            'Stock' : [],
            'isNew' : [],
            'fullPrice' : [],
            'subtitle' : [],
            'current_price':[],
            'title' : []
}
    
for product in data2:
        Data_products['id'].append(product['id'])
        Data_products['colorDescription'].append(product['colorDescription'])
        Data_products['portrait_url'].append(product['images']['portraitURL'])
        Data_products['squarishURL'].append(product['images']['squarishURL'])
        Data_products['Stock'].append(product['inStock'])
        Data_products['isNew'].append(product['colorways'][0]['isNew'])
        Data_products['fullPrice'].append(product['price']['fullPrice'])
        Data_products[ 'current_price'].append(product['price']['currentPrice'])
        Data_products['subtitle'].append(product['subtitle'])
        Data_products['title'].append(product['title'])

In [65]:
Data_2 = pd.DataFrame(Data_products)

In [66]:
f = Data_ != Data_2


ValueError: Can only compare identically-labeled DataFrame objects

In [67]:
f.loc[f['id'] ==  True]

NameError: name 'f' is not defined

In [68]:
Data_.tail()

,id,colorDescription,portrait_url,squarishURL,Stock,isNew,fullPrice,subtitle,current_price,title
571,f8c723cc-04a4-3feb-aa11-68d88a0fc087,Azul vacío/Blanco/Negro/Voltio ligero,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/ogn...,True,False,1798,Calzado de fútbol para terreno blando,1438,Nike Mercurial Vapor 13 Academy MDS SG-PRO Ant...
572,28fc2d9c-7f94-39f1-bd2f-20504d254d69,Mango brillante/Naranja pulso/Antracita/Blanco,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/zpr...,True,False,2298,Calzado de fútbol para cancha cubierta,1838,Nike Zoom Phantom Venom Pro IC
573,b1d5abfa-3708-3e42-900d-46b83342a8ec,Volt/Volt/Blanco,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/twj...,True,False,1898,Calzado de fútbol para cancha cubierta,1518,Nike Phantom Vision Academy Dynamic Fit IC
574,d1f94fb9-a50d-3065-a31f-1a4bbb08a56a,Negro/Oro metalizado/Gris cemento,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/upe...,False,False,4799,Calzado para hombre,3838,Air Jordan 3 Retro Tinker
575,57c95a96-16bd-3bb4-af0f-b489770f8f51,Volt/Voltio ligero/Blanco,"https://static.nike.com/a/images/ar_4:5,c_pad,...",https://static.nike.com/a/images/t_default/zha...,True,False,5698,Calzado de fútbol SG-PRO,4558,Nike Phantom Vision Elite Dynamic Fit Anti-Clo...


In [69]:
Data_2.tail()

,id,colorDescription,portrait_url,squarishURL,Stock,isNew,fullPrice,subtitle,current_price,title
157,ec798b89-6f68-3b0c-88a8-d13f80f05aae,Blanco/Platino puro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/w3j...,True,False,2699,Calzado para hombre,2158,Nike React Element 55 SE
158,d4db046a-ff29-303f-aad9-5e359cd00d36,Azul vacío/Blanco/Negro/Voltio ligero,"https://static.nike.com/a/images/ar_4:5,c_pad,...",https://static.nike.com/a/images/t_default/n1x...,True,False,5498,Calzado de fútbol para terreno blando,4398,Nike Mercurial Vapor 13 Elite MDS SG-PRO Anti-...
159,767da377-2a2d-3713-b437-10dbbc092de5,Obsidiana/Blanco,"https://c.static-nike.com/a/images/ar_4:5,c_pa...",https://c.static-nike.com/a/images/t_default/f...,True,False,1549,Calzado unisex,1238,Nike Classic Cortez Nylon
160,b290626a-e1c7-34c1-9dfd-ed3004ae98c7,Negro/Negro,"https://c.static-nike.com/a/images/c_limit,w_4...",https://c.static-nike.com/a/images/t_default/c...,True,False,1399,Calzado para hombre,1118,Nike Viale
161,d1f94fb9-a50d-3065-a31f-1a4bbb08a56a,Negro/Oro metalizado/Gris cemento,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/upe...,False,False,4799,Calzado para hombre,3838,Air Jordan 3 Retro Tinker


In [70]:
df_diff = pd.concat([Data_,Data_2]).drop_duplicates(subset = 'title',keep=False)
df_diff.head()

,id,colorDescription,portrait_url,squarishURL,Stock,isNew,fullPrice,subtitle,current_price,title
302,3321eee1-85dc-3c44-bfe8-ba44e621e330,Negro/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/9bb...,True,False,1798,Calzado de fútbol para terreno blando,1618,Nike Tiempo Legend 8 Academy SG-PRO Anti-Clog ...
16,c50fafb3-3f08-3718-b2a2-b66d8d6db461,Negro/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/97b...,True,False,1698,Calzado de fútbol para múltiples superficies,1608,Nike Tiempo Legend 8 Academy MG
70,2e48d5ec-6f0e-38cb-b007-60e5d9ba6dc0,Carmesí láser/Negro/Plata metalizado,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/7a2...,True,False,2998,Calzado de fútbol para terreno firme,2398,Nike Phantom Vision 2 Pro Dynamic Fit FG


In [71]:
df_diff

,id,colorDescription,portrait_url,squarishURL,Stock,isNew,fullPrice,subtitle,current_price,title
302,3321eee1-85dc-3c44-bfe8-ba44e621e330,Negro/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/9bb...,True,False,1798,Calzado de fútbol para terreno blando,1618,Nike Tiempo Legend 8 Academy SG-PRO Anti-Clog ...
16,c50fafb3-3f08-3718-b2a2-b66d8d6db461,Negro/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/97b...,True,False,1698,Calzado de fútbol para múltiples superficies,1608,Nike Tiempo Legend 8 Academy MG
70,2e48d5ec-6f0e-38cb-b007-60e5d9ba6dc0,Carmesí láser/Negro/Plata metalizado,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/7a2...,True,False,2998,Calzado de fútbol para terreno firme,2398,Nike Phantom Vision 2 Pro Dynamic Fit FG


In [35]:
Data_['test'] = Data_['fullPrice']-Data_['current_price']

In [36]:
Data_.loc[Data_['test']> 1000]

,id,colorDescription,portrait_url,squarishURL,Stock,isNew,fullPrice,subtitle,current_price,title,test
19,21f6fe89-a5e8-38e4-86d7-348f5b00644c,Vela/Transparente/Vela,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/f20...,True,False,3549,Calzado para hombre,2478,Nike Air Max Tailwind 99,1071
20,d4b3bfa5-098d-3e0a-ba98-3a8a29bf4166,Gris hierro/Negro/Blanco/Cerúleo,"https://static.nike.com/a/images/ar_4:5,c_pad,...",https://static.nike.com/a/images/t_default/bea...,True,False,3449,Calzado para hombre,2408,Nike Air Max Tailwind IV,1041
21,8e03f418-a2c3-3bed-8913-8b783b104c4e,Sombrío Apagado/Negro/Antracita/Azul verdoso m...,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/zz0...,True,False,3399,Calzado para hombre,2378,Nike Air Max Tailwind IV,1021
31,547b3a23-694c-3738-b2c5-e380abeaaa9d,Gris azulado/Verde eléctrico/Negro/Volt,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/gtc...,True,False,3699,Calzado para hombre,2588,Nike Air Max 720,1111
35,a8b00bb2-50b5-3eb7-9271-245bd3c6c45a,Negro/Antracita/Blanco,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/sfs...,True,False,3399,Calzado para hombre,2378,Nike Air Max 270 React,1021
39,35fd5b93-89a5-3288-ba07-95df12595d1d,Hueso claro/Gris azulado/Vela/Plata metalizado,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/k2q...,True,False,3399,Calzado para hombre,2378,Nike Shox TL,1021
41,13f67f22-b74e-3d7f-83d0-05d745767d80,Veneno limón/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/i1-...,True,False,6398,Calzado de fútbol para césped artificial,5118,Nike Mercurial Superfly 7 Elite MDS AG-PRO,1280
44,8f3cb2cd-43bb-3920-806c-a9db618bd8a2,Azul vacío/Blanco/Negro/Voltio ligero,"https://static.nike.com/a/images/ar_4:5,c_pad,...",https://static.nike.com/a/images/t_default/72a...,True,False,6098,Calzado de fútbol para terreno blando,4878,Nike Mercurial Superfly 7 Elite MDS SG-PRO Ant...,1220
48,2a9e979d-148e-3f59-b4fa-d11f9c4c3bd4,Veneno limón/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/i1-...,True,False,5698,Calzado de fútbol para césped artificial,4558,Nike Mercurial Vapor 13 Elite MDS AG-PRO,1140
51,1a9bb07b-e53d-3926-853d-5bbf01e804bf,Negro/Oro intenso metálico,"https://c.static-nike.com/a/images/c_limit,w_4...",https://c.static-nike.com/a/images/t_default/h...,True,False,5499,Calzado de fútbol para terreno firme,4398,Nike Phantom Vision Elite Dynamic Fit FG,1101


In [37]:
Data_

,id,colorDescription,portrait_url,squarishURL,Stock,isNew,fullPrice,subtitle,current_price,title,test
0,ddf87082-4fe7-3579-ac8c-bea6e1880e6c,Negro/Blanco/Azul universitario/Naranja láser,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/d84...,True,False,3499,Calzado de running para hombre,2798,Nike Joyride Run Flyknit,701
1,6a447f28-2ad8-315b-bcbf-898f929781d8,Blanco cumbre/Antracita/Platino puro/Azul univ...,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/qvz...,True,False,2698,Calzado de golf para hombre,2158,Nike Air Max 1 G,540
2,b844f3de-ae38-35e5-a8b1-18f88da2e479,Negro/Blanco,"https://c.static-nike.com/a/images/c_limit,w_4...",https://c.static-nike.com/a/images/t_default/o...,True,False,2599,Calzado para hombre,2078,Nike React Element 55,521
3,7dc2229a-0231-3071-8144-fd32d0ef0db0,Blanco/Carmesí brillante/Oro universitario/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/qm9...,True,False,2299,Calzado para hombre,1838,Nike Air Max 200 (2000 World Stage),461
4,9c2e7747-5dc9-3b3b-a83c-1e169542f14b,Gris oscuro/Cubo océano/Blanco cumbre/Blanco,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/md6...,True,False,2599,Calzado,2078,Nike Signal D/MS/X,521
5,2bbae96e-bd7c-3ce3-b161-dd0245df22cb,Negro/Rojo órbita/Violeta enérgico/Gris oscuro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/yr7...,True,False,3199,Calzado para hombre,2238,Nike N110 D/MS/X,961
6,04628377-82c2-35e4-9770-daa5034dfbe9,Negro/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/ss3...,False,False,1999,Calzado,1598,Nike P-6000,401
7,9dfddea7-7b7d-3fad-b9a0-616ead36fa3b,Negro/Azul héroe/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/uxg...,True,False,1649,Calzado de fútbol para cancha cubierta,1318,Nike Tiempo Legend 8 Academy IC,331
8,17800505-3120-3a43-a6cd-e22832e7742a,Carmesí láser/Carmesí láser/Negro,"https://static.nike.com/a/images/c_limit,w_400...",https://static.nike.com/a/images/t_default/5c7...,True,False,1699,Calzado de fútbol para múltiples superficies,1528,Nike Mercurial Vapor 13 Academy MG,171
9,9e8b6c5e-3166-3a07-9a9a-47f69b7c9842,Negro/Naranja máximo/Plata metalizado,"https://c.static-nike.com/a/images/ar_4:5,c_pa...",https://c.static-nike.com/a/images/t_default/z...,True,False,2649,Calzado,2118,Nike Shox R4,531
